In [ ]:
import requests
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up Spotify API client
client_id = "5efcafaa1ddf4fb2a2808bce841c1b88"  # Replace with your Spotify Client ID
client_secret = "0735f01a14c84c688504bca3367167f8"  # Replace with your Spotify Client Secret

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# TMDb API Setup
TMDB_API_KEY = '5ab369b78ced207af8e34061d64d9c8d'  # Replace with your TMDb API key
TMDB_BASE_URL = 'https://api.themoviedb.org/3'

# Load pre-trained model and tokenizer
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

print("AI Chatbot: Hi! Type 'exit' to end the conversation.")

# Function to get movies by genre
def get_movies_by_genre(genre_id, page=1):
    url = f"{TMDB_BASE_URL}/discover/movie?api_key={TMDB_API_KEY}&with_genres={genre_id}&page={page}"
    response = requests.get(url)
    data = response.json()
    if data.get('results'):
        movies = []
        for movie in data['results']:
            movies.append(f"Title: {movie['title']}, Release Date: {movie['release_date']}, Overview: {movie['overview']}")
        return movies
    else:
        return ["No movies found for this genre."]

# Function to get movies by actor (returns just the first movie for simplicity)
def get_movies_by_actor(actor_name, recommended_movies):
    try:
        search_url = f"{TMDB_BASE_URL}/search/person?api_key={TMDB_API_KEY}&query={actor_name}"
        search_response = requests.get(search_url)
        search_data = search_response.json()

        if search_data.get('results'):
            actor_id = search_data['results'][0]['id']
            actor_movies_url = f"{TMDB_BASE_URL}/person/{actor_id}/movie_credits?api_key={TMDB_API_KEY}"
            actor_movies_response = requests.get(actor_movies_url)
            actor_movies_data = actor_movies_response.json()

            # Return just the first movie, if not already recommended
            for movie in actor_movies_data['cast']:
                if movie['title'] not in recommended_movies:
                    recommended_movies.append(movie['title'])
                    return movie['title'], recommended_movies

            return "No more new movies found for this actor.", recommended_movies
        else:
            return "No movies found for this actor.", recommended_movies
    except Exception as e:
        return f"Error fetching movies by actor: {e}", recommended_movies

# Function to get a list of song recommendations from a specific artist
def get_artist_song_recommendations(artist_name, already_recommended):
    print(f"Searching for songs by {artist_name}...")
    results = sp.search(q=f'artist:{artist_name}', type='track', limit=10)

    songs = [song for song in results['tracks']['items'] if song['name'] not in already_recommended]

    if songs:
        song = songs[0]
        song_name = song['name']
        artist_name = song['artists'][0]['name']
        already_recommended.append(song_name)
        return f"How about listening to '{song_name}' by {artist_name}?", already_recommended
    else:
        return f"Sorry, I've run out of new songs by {artist_name}. Try another artist!", already_recommended

# Function to suggest songs based on the user's mood using Spotify search
def get_song_based_on_mood(mood):
    mood = mood.lower()

    # Search moods related keywords or phrases to match the mood
    mood_keywords = {
        "happy": ["happy", "joy", "upbeat", "cheerful"],
        "sad": ["sad", "melancholy", "lonely", "blue"],
        "energetic": ["energetic", "workout", "party", "dance"],
        "chill": ["chill", "relaxed", "calm", "easy"]
    }

    # Search Spotify for songs that match the mood keywords
    if mood in mood_keywords:
        search_results = []
        for keyword in mood_keywords[mood]:
            search_results += sp.search(q=keyword, type='track', limit=5)['tracks']['items']

        if search_results:
            song = search_results[0]
            song_name = song['name']
            artist_name = song['artists'][0]['name']
            return f"How about listening to '{song_name}' by {artist_name}? It's perfect for your mood!", []
        else:
            return f"Sorry, I couldn't find any songs for the mood '{mood}'.", []
    else:
        return "Sorry, I don't have song recommendations for that mood right now. Try a different mood!", []

# Predefined responses to improve conversation flow
def handle_stressful_day(user_input):
    stressful_keywords = ["stress", "tired", "work", "busy", "exhausted"]
    if any(keyword in user_input for keyword in stressful_keywords):
        return "I'm really sorry to hear that! It sounds like you've had a tough day. Do you want to talk about it or listen to some relaxing music?"
    return None

# Chat history storage
chat_history_ids = None
already_recommended_songs = []  # To store recommended songs to avoid repetition
recommended_movies = []  # To store recommended movies to avoid repetition

while True:
    user_input = input("You: ").lower()

    if user_input == "exit":
        print("AI Chatbot: Bye! Have a nice day.")
        break

    if "change the subject" in user_input:
        print("AI Chatbot: Sure, let's talk about something else! I'm listening.")
        chat_history_ids = None  # Clear the chat history
        continue

    stress_response = handle_stressful_day(user_input)
    if stress_response:
        print(f"AI Chatbot: {stress_response}")
        continue  # Skip DialoGPT response

    # Check if the user asks for a song recommendation by a specific artist (e.g., "suggest me a song by Shreya Ghosal")
    if "suggest me a song by" in user_input:
        artist_name = user_input.replace("suggest me a song by", "").strip()
        if artist_name:
            song_recommendation, already_recommended_songs = get_artist_song_recommendations(artist_name, already_recommended_songs)
            print(f"AI Chatbot: {song_recommendation}")
        else:
            print("AI Chatbot: Please specify an artist for the song you'd like!")
        continue  # Skip DialoGPT response

    # Check if the user asks for a song recommendation based on mood (e.g., "suggest me happy song")
    if "suggest me" in user_input and "song" in user_input and "by" not in user_input:
        mood = user_input.replace("suggest me", "").replace("song", "").strip()
        if mood:
            song_recommendation, _ = get_song_based_on_mood(mood)
            print(f"AI Chatbot: {song_recommendation}")
        else:
            print("AI Chatbot: Please specify the mood for the song you'd like!")
        continue  # Skip DialoGPT response

    # Check if the user asks for a movie recommendation (i.e., "suggest me a movie starring")
    if "suggest me a movie starring" in user_input:
        actor_name = user_input.replace("suggest me a movie starring", "").strip()
        if actor_name:
            movie, recommended_movies = get_movies_by_actor(actor_name, recommended_movies)  # Get just one movie
            print(f"AI Chatbot: How about watching '{movie}'?")
        else:
            print("AI Chatbot: Please specify an actor you'd like movie recommendations for!")
        continue  # Skip DialoGPT response

    # Tokenize input and encode the user's input for conversational purposes
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append the new input to the chat history (if exists)
    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
    else:
        bot_input_ids = new_input_ids

    # Generate response using DialoGPT
    chat_history_ids = model.generate(
        bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id
    )

    # Decode and print the bot's response
    bot_response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print(f"AI Chatbot: {bot_response}")


C:\Users\PC\AppData\Local\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\PC\AppData\Local\anaconda3\Lib\site-packages\transformers\modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.seriali

AI Chatbot: Hi! Type 'exit' to end the conversation.
You: hiiiii


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


AI Chatbot: hiya :D
You: hii


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

AI Chatbot: I don't. I just know you.
You: Brooooo


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


AI Chatbot: I'm not your bro, bro.
You: suggest me a song by arijit singh
Searching for songs by arijit singh...
AI Chatbot: How about listening to 'Tujhe Kitna Chahne Lage (From "Kabir Singh")' by Arijit Singh?
You: suggest me a movie starring by Sidharth malhotra
AI Chatbot: How about watching 'Yodha'?
You: you are the best 


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


AI Chatbot: I'm not your bro, bro.
